In [2]:
import wsbcatalog, rdflib
import numpy as np, pandas as pd, rdflib, re, collections
from rdflib.namespace import RDF, RDFS
from rdflib import URIRef, BNode, Literal

In [3]:
catalog = wsbcatalog.WSBCatalog()

In [4]:
raw_items = catalog.get_items()[['identifier', 'bibliographicCitation', 'description', 'image', 'thumbnail']]

In [5]:
raw_items['itemOf'] = raw_items['bibliographicCitation'].str.split(', ')

/Users/bradleyallen/anaconda3/envs/wsbcat/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
items = raw_items[['identifier', 'itemOf', 'description', 'image', 'thumbnail']].to_dict('records')

In [7]:
abri = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/4.0/instance/")
abrw = rdflib.Namespace("https://wsburroughs.link/anything-but-routine/4.0/work/")
bf = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
arm = rdflib.Namespace("https://w3id.org/arm/core/ontology/0.1/")
bpacol = rdflib.Namespace("https://wsburroughs.link/bradleypallen/item/")

In [8]:
def initialize_abr_graph():
    g = rdflib.Graph()
    g.bind("abri", "https://wsburroughs.link/anything-but-routine/4.0/instance/")
    g.bind("abrw", "https://wsburroughs.link/anything-but-routine/4.0/work/")
    g.bind("bf", "http://id.loc.gov/ontologies/bibframe/")
    g.bind("arm", "https://w3id.org/arm/core/ontology/0.1/")
    g.bind("bpacol", "https://wsburroughs.link/bradleypallen/item/")
    return g

### Items

In [16]:
for item in items:
    g = initialize_abr_graph()
    name = item['identifier']
    id = bpacol[name]
    g.add((id, RDF.type, bf.Item))
    g.add((id, RDF.type, bf.Text))
    # bf:heldBy
    bpa = BNode()
    g.add((id, bf.heldBy, bpa))
    g.add((bpa, RDF.type, bf.Agent))
    g.add((bpa, RDF.type, bf.Person)) 
    g.add((bpa, bf.role, Literal("collector")))
    g.add((bpa, RDFS.label, Literal("Bradley P. Allen")))
    # bf:note
    # todo: parse description for arm:LimitationNote, etc.
    note = BNode()
    g.add((id, bf.note, note))
    g.add((note, RDF.type, bf.Note))
    g.add((note, RDF.type, arm.SubjectNote))
    g.add((note, RDF.value, Literal(item['description'])))
    # bf:itemOf
    for inst in item['itemOf']:
        if re.match('schottlaender', inst):
            g.add((id, bf.itemOf, abri[inst.split('/')[-1]]))
    g.serialize(f"item/{name}.ttl", format='turtle')